In [1]:
import random
import os
import re
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
import seaborn as sns
import pickle
import torch

pd.set_option('display.expand_frame_repr', False)  # DataFrameを改行せずに表示
pd.set_option('display.max_columns', None)  # すべての列を表示

In [2]:
def extract_finalQP(filename):
    match = re.search(r'2ndQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def extract_1stQP(filename):
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None

def is_double_compressed(mean_difference, final_QP, threshold):    
    mean_difference = mean_difference[0]
    final_QP = final_QP[0]

    energy_clamp = torch.clamp(mean_difference, min=0)
    energy = torch.sum(torch.square(energy_clamp))
    mean_difference_right_clamp = torch.clamp(mean_difference[final_QP+1:52], min=0)
    right_energy = torch.sum(torch.square(mean_difference_right_clamp))
    
    if energy != 0:
        energy_ratio = right_energy / energy
        if energy_ratio >= threshold:
            return True
        elif energy_ratio < threshold:
            return False
    else:
        # energyが0の場合、エラーを処理するか、適切な値を返す
        return -1

    # if (right_energy / energy) != 0 and (right_energy / energy) >= threshold:
    #     return True
    # elif (right_energy / energy) != 0 and (right_energy / energy) < threshold:
    #     return False
    # else:
    #     return -1
    
def calculate_mae(file_path):
    try:
        with open(file_path, 'rb') as file:
            loaded_data, loaded_data_shifted = pickle.load(file)
    except Exception as e:
        print(f"Error occurred while loading {file_path}: {e}")
        return None

    # タプル内のリストを抽出
    original_mae = loaded_data
    shifted_mae = loaded_data_shifted

    # Coding ghostを計算してリストに格納する
    mae_difference = [shifted - original for original, shifted in zip(original_mae, shifted_mae)]
    
    # mae_differenceをtensorに変換
    mae_difference_tensor = torch.tensor(mae_difference)
    
    # mae_differenceの各要素においてマイナスの値を0に変換
    mae_difference_positive = [0 if val <= 0 else val for val in mae_difference]
    
    return mae_difference_positive, mae_difference_tensor

In [3]:
rootpath = "/Prove/Yoshihisa/HEIF_ghost/HEIF_IMAGES_CSV/"

single_train_csv_path1 = os.path.join(rootpath, 'HEIF_images_single_csv')
single_train_csv_path2 = os.path.join(rootpath, 'HEIF_images_second_sameQP_csv')

second_train_csv_path1 = os.path.join(rootpath, 'HEIF_images_second_csv')
second_train_csv_path2 = os.path.join(rootpath, 'HEIF_images_triple_csv')


single_train_csv_path1_list = [os.path.join(single_train_csv_path1, file) for file in sorted(os.listdir(single_train_csv_path1))]
single_train_csv_path2_list = [os.path.join(single_train_csv_path2, file) for file in sorted(os.listdir(single_train_csv_path2))]

second_train_csv_path1_list = [os.path.join(second_train_csv_path1, file) for file in sorted(os.listdir(second_train_csv_path1))]
second_train_csv_path2_list = [os.path.join(second_train_csv_path2, file) for file in sorted(os.listdir(second_train_csv_path2))]


print("single_train_csv_path1_list: ", len(single_train_csv_path1_list))
print("single_train_csv_path2_list: ", len(single_train_csv_path2_list))

print("second_train_csv_path1_list: ", len(second_train_csv_path1_list))
print("second_train_csv_path2_list: ", len(second_train_csv_path2_list))


single_train_csv_path1_list:  3080
single_train_csv_path2_list:  3080
second_train_csv_path1_list:  17556
second_train_csv_path2_list:  17556


In [4]:
second_train_csv_path1 = os.path.join(rootpath, 'HEIF_images_second_sameQP_csv')
second_train_csv_path2 = os.path.join(rootpath, 'HEIF_images_triple_sameQP_csv')

second_sameQP_train_csv_path1_list = [os.path.join(second_train_csv_path1, file) for file in sorted(os.listdir(second_train_csv_path1))]
second_sameQP_train_csv_path2_list = [os.path.join(second_train_csv_path2, file) for file in sorted(os.listdir(second_train_csv_path2))]

print("second_sameQP_train_csv_path1_list: ", len(second_sameQP_train_csv_path1_list))
print("second_sameQP_train_csv_path2_list: ", len(second_sameQP_train_csv_path2_list))

second_sameQP_train_csv_path1_list:  3080
second_sameQP_train_csv_path2_list:  3080


In [5]:
second_train_csv_path1 = os.path.join(rootpath, 'HEIF_images_second_largeQP_csv')
second_train_csv_path2 = os.path.join(rootpath, 'HEIF_images_triple_largeQP_csv')

second_largeQP_train_csv_path1_list = [os.path.join(second_train_csv_path1, file) for file in sorted(os.listdir(second_train_csv_path1))]
second_largeQP_train_csv_path2_list = [os.path.join(second_train_csv_path2, file) for file in sorted(os.listdir(second_train_csv_path2))]

print("second_largeQP_train_csv_path1_list: ", len(second_largeQP_train_csv_path1_list))
print("second_largeQP_train_csv_path2_list: ", len(second_largeQP_train_csv_path2_list))

second_largeQP_train_csv_path1_list:  12012
second_largeQP_train_csv_path2_list:  12012


In [6]:
rootpath = "/Prove/Yoshihisa/HEIF_ghost/PKL/"

single_train_pkl_path1 = os.path.join(rootpath, 'pkl_single')
single_train_pkl_path2 = os.path.join(rootpath, 'pkl_second_sameQP')

second_train_pkl_path1 = os.path.join(rootpath, 'pkl_second')
second_train_pkl_path2 = os.path.join(rootpath, 'pkl_triple')


single_train_pkl_path1_list = [os.path.join(single_train_pkl_path1, file) for file in sorted(os.listdir(single_train_pkl_path1))]
single_train_pkl_path2_list = [os.path.join(single_train_pkl_path2, file) for file in sorted(os.listdir(single_train_pkl_path2))]

second_train_pkl_path1_list = [os.path.join(second_train_pkl_path1, file) for file in sorted(os.listdir(second_train_pkl_path1))]
second_train_pkl_path2_list = [os.path.join(second_train_pkl_path2, file) for file in sorted(os.listdir(second_train_pkl_path2))]


print("single_train_pkl_path1_list: ", len(single_train_pkl_path1_list))
print("single_train_pkl_path2_list: ", len(single_train_pkl_path2_list))

print("second_train_pkl_path1_list: ", len(second_train_pkl_path1_list))
print("second_train_pkl_path2_list: ", len(second_train_pkl_path2_list))


single_train_pkl_path1_list:  3080
single_train_pkl_path2_list:  3080
second_train_pkl_path1_list:  17556
second_train_pkl_path2_list:  17556


In [7]:
second_train_pkl_path1 = os.path.join(rootpath, 'pkl_second_sameQP')
second_train_pkl_path2 = os.path.join(rootpath, 'pkl_triple_sameQP')

second_sameQP_train_pkl_path1_list = [os.path.join(second_train_pkl_path1, file) for file in sorted(os.listdir(second_train_pkl_path1))]
second_sameQP_train_pkl_path2_list = [os.path.join(second_train_pkl_path2, file) for file in sorted(os.listdir(second_train_pkl_path2))]

print("second_sameQP_train_pkl_path1_list: ", len(second_sameQP_train_pkl_path1_list))
print("second_sameQP_train_pkl_path2_list: ", len(second_sameQP_train_pkl_path2_list))

second_sameQP_train_pkl_path1_list:  3080
second_sameQP_train_pkl_path2_list:  3080


In [8]:
second_train_pkl_path1 = os.path.join(rootpath, 'pkl_second_largeQP')
second_train_pkl_path2 = os.path.join(rootpath, 'pkl_triple_largeQP')

second_largeQP_train_pkl_path1_list = [os.path.join(second_train_pkl_path1, file) for file in sorted(os.listdir(second_train_pkl_path1))]
second_largeQP_train_pkl_path2_list = [os.path.join(second_train_pkl_path2, file) for file in sorted(os.listdir(second_train_pkl_path2))]

print("second_largeQP_train_pkl_path1_list: ", len(second_largeQP_train_pkl_path1_list))
print("second_largeQP_train_pkl_path2_list: ", len(second_largeQP_train_pkl_path2_list))

second_largeQP_train_pkl_path1_list:  12012
second_largeQP_train_pkl_path2_list:  12012


In [9]:
single_train_csv = list(zip(single_train_csv_path1_list, single_train_pkl_path1_list, single_train_csv_path2_list, single_train_pkl_path2_list))

second_train_csv = list(zip(second_train_csv_path1_list, second_train_pkl_path1_list, second_train_csv_path2_list, second_train_pkl_path2_list))
second_sameQP_train_csv = list(zip(second_sameQP_train_csv_path1_list, second_sameQP_train_pkl_path1_list, second_sameQP_train_csv_path2_list, second_sameQP_train_pkl_path2_list))
second_largeQP_train_csv = list(zip(second_largeQP_train_csv_path1_list, second_largeQP_train_pkl_path1_list, second_largeQP_train_csv_path2_list, second_largeQP_train_pkl_path2_list))

single_train_csv = random.sample(single_train_csv, 3000)
second_train_csv = random.sample(second_train_csv, 1000)
second_sameQP_train_csv = random.sample(second_sameQP_train_csv, 1000)
second_largeQP_train_csv = random.sample(second_largeQP_train_csv, 1000)

train_csv_list = single_train_csv + second_train_csv + second_sameQP_train_csv + second_largeQP_train_csv

print("train_csv_list: ", len(train_csv_list))

train_csv_list:  6000


In [10]:
rootpath2 = "/Prove/Yoshihisa/HEIF_ghost/EXPERIMENT_DIFFERENT_SOFTWARE/"

# SINGLE
GIMP_path1 = os.path.join(rootpath2, 'GIMP_csv')
GIMP_path2 = os.path.join(rootpath2, 'GIMP_RECOMPRESSED_csv')

GIMP_path1_csv = [os.path.join(GIMP_path1, file) for file in sorted(os.listdir(GIMP_path1))]
GIMP_path2_csv = [os.path.join(GIMP_path2, file) for file in sorted(os.listdir(GIMP_path2))]

LIBHEIF_path1 = os.path.join(rootpath2, 'LIBHEIF_csv')
LIBHEIF_path2 = os.path.join(rootpath2, 'LIBHEIF_RECOMPRESSED_csv')

LIBHEIF_path1_csv = [os.path.join(LIBHEIF_path1, file) for file in sorted(os.listdir(LIBHEIF_path1))]
LIBHEIF_path2_csv = [os.path.join(LIBHEIF_path2, file) for file in sorted(os.listdir(LIBHEIF_path2))]


# DOUBLE
GIMP_GIMP_path1 = os.path.join(rootpath2, 'GIMP_GIMP_csv')
GIMP_GIMP_path2 = os.path.join(rootpath2, 'GIMP_GIMP_RECOMPRESSED_csv')

GIMP_GIMP_path1_csv = [os.path.join(GIMP_GIMP_path1, file) for file in sorted(os.listdir(GIMP_GIMP_path1))]
GIMP_GIMP_path2_csv = [os.path.join(GIMP_GIMP_path2, file) for file in sorted(os.listdir(GIMP_GIMP_path2))]

LIBHEIF_GIMP_path1 = os.path.join(rootpath2, 'LIBHEIF_GIMP_csv')
LIBHEIF_GIMP_path2 = os.path.join(rootpath2, 'LIBHEIF_GIMP_RECOMPRESSED_csv')

LIBHEIF_GIMP_path1_csv = [os.path.join(LIBHEIF_GIMP_path1, file) for file in sorted(os.listdir(LIBHEIF_GIMP_path1))]
LIBHEIF_GIMP_path2_csv = [os.path.join(LIBHEIF_GIMP_path2, file) for file in sorted(os.listdir(LIBHEIF_GIMP_path2))]

GIMP_LIBHEIF_path1 = os.path.join(rootpath2, 'GIMP_LIBHEIF_csv')
GIMP_LIBHEIF_path2 = os.path.join(rootpath2, 'GIMP_LIBHEIF_RECOMPRESSED_csv')

GIMP_LIBHEIF_path1_csv = [os.path.join(GIMP_LIBHEIF_path1, file) for file in sorted(os.listdir(GIMP_LIBHEIF_path1))]
GIMP_LIBHEIF_path2_csv = [os.path.join(GIMP_LIBHEIF_path2, file) for file in sorted(os.listdir(GIMP_LIBHEIF_path2))]


print("GIMP_path1_csv: ", len(GIMP_path1_csv))
print("GIMP_path2_csv: ", len(GIMP_path2_csv))
print("GIMP_GIMP_path1_csv: ", len(GIMP_GIMP_path1_csv))
print("GIMP_GIMP_path2_csv: ", len(GIMP_GIMP_path2_csv))
print()
print("GIMP_path1_csv: ", len(GIMP_path1_csv))
print("GIMP_path2_csv: ", len(GIMP_path2_csv))
print("LIBHEIF_GIMP_path1_csv: ", len(LIBHEIF_GIMP_path1_csv))
print("LIBHEIF_GIMP_path2_csv: ", len(LIBHEIF_GIMP_path2_csv))
print()
print("LIBHEIF_path1_csv: ", len(LIBHEIF_path1_csv))
print("LIBHEIF_path2_csv: ", len(LIBHEIF_path2_csv))
print("GIMP_LIBHEIF_path1_csv: ", len(GIMP_LIBHEIF_path1_csv))
print("GIMP_LIBHEIF_path2_csv: ", len(GIMP_LIBHEIF_path2_csv))


GIMP_path1_csv:  90
GIMP_path2_csv:  90
GIMP_GIMP_path1_csv:  90
GIMP_GIMP_path2_csv:  90

GIMP_path1_csv:  90
GIMP_path2_csv:  90
LIBHEIF_GIMP_path1_csv:  90
LIBHEIF_GIMP_path2_csv:  90

LIBHEIF_path1_csv:  90
LIBHEIF_path2_csv:  90
GIMP_LIBHEIF_path1_csv:  90
GIMP_LIBHEIF_path2_csv:  90


In [11]:
rootpath3 = "/Prove/Yoshihisa/HEIF_ghost/EXPERIMENT_DIFFERENT_SOFTWARE/PKL/"

# SINGLE
GIMP_path1 = os.path.join(rootpath3, 'pkl_GIMP')
GIMP_path2 = os.path.join(rootpath3, 'pkl_GIMP_RECOMPRESSED')

GIMP_path1_pkl = [os.path.join(GIMP_path1, file) for file in sorted(os.listdir(GIMP_path1))]
GIMP_path2_pkl = [os.path.join(GIMP_path2, file) for file in sorted(os.listdir(GIMP_path2))]

LIBHEIF_path1 = os.path.join(rootpath3, 'pkl_LIBHEIF')
LIBHEIF_path2 = os.path.join(rootpath3, 'pkl_LIBHEIF_RECOMPRESSED')

LIBHEIF_path1_pkl = [os.path.join(LIBHEIF_path1, file) for file in sorted(os.listdir(LIBHEIF_path1))]
LIBHEIF_path2_pkl = [os.path.join(LIBHEIF_path2, file) for file in sorted(os.listdir(LIBHEIF_path2))]


# DOUBLE
GIMP_GIMP_path1 = os.path.join(rootpath3, 'pkl_GIMP_GIMP')
GIMP_GIMP_path2 = os.path.join(rootpath3, 'pkl_GIMP_GIMP_RECOMPRESSED')

GIMP_GIMP_path1_pkl = [os.path.join(GIMP_GIMP_path1, file) for file in sorted(os.listdir(GIMP_GIMP_path1))]
GIMP_GIMP_path2_pkl = [os.path.join(GIMP_GIMP_path2, file) for file in sorted(os.listdir(GIMP_GIMP_path2))]

LIBHEIF_GIMP_path1 = os.path.join(rootpath3, 'pkl_LIBHEIF_GIMP')
LIBHEIF_GIMP_path2 = os.path.join(rootpath3, 'pkl_LIBHEIF_GIMP_RECOMPRESSED')

LIBHEIF_GIMP_path1_pkl = [os.path.join(LIBHEIF_GIMP_path1, file) for file in sorted(os.listdir(LIBHEIF_GIMP_path1))]
LIBHEIF_GIMP_path2_pkl = [os.path.join(LIBHEIF_GIMP_path2, file) for file in sorted(os.listdir(LIBHEIF_GIMP_path2))]

GIMP_LIBHEIF_path1 = os.path.join(rootpath3, 'pkl_GIMP_LIBHEIF')
GIMP_LIBHEIF_path2 = os.path.join(rootpath3, 'pkl_GIMP_LIBHEIF_RECOMPRESSED')

GIMP_LIBHEIF_path1_pkl = [os.path.join(GIMP_LIBHEIF_path1, file) for file in sorted(os.listdir(GIMP_LIBHEIF_path1))]
GIMP_LIBHEIF_path2_pkl = [os.path.join(GIMP_LIBHEIF_path2, file) for file in sorted(os.listdir(GIMP_LIBHEIF_path2))]


print("GIMP_path1_pkl: ", len(GIMP_path1_pkl))
print("GIMP_path2_pkl: ", len(GIMP_path2_pkl))
print("GIMP_GIMP_path1_pkl: ", len(GIMP_GIMP_path1_pkl))
print("GIMP_GIMP_path2_pkl: ", len(GIMP_GIMP_path2_pkl))
print()
print("GIMP_path1_pkl: ", len(GIMP_path1_pkl))
print("GIMP_path2_pkl: ", len(GIMP_path2_pkl))
print("LIBHEIF_GIMP_path1_pkl: ", len(LIBHEIF_GIMP_path1_pkl))
print("LIBHEIF_GIMP_path2_pkl: ", len(LIBHEIF_GIMP_path2_pkl))
print()
print("LIBHEIF_path1_pkl: ", len(LIBHEIF_path1_pkl))
print("LIBHEIF_path2_pkl: ", len(LIBHEIF_path2_pkl))
print("GIMP_LIBHEIF_path1_pkl: ", len(GIMP_LIBHEIF_path1_pkl))
print("GIMP_LIBHEIF_path2_pkl: ", len(GIMP_LIBHEIF_path2_pkl))


GIMP_path1_pkl:  90
GIMP_path2_pkl:  90
GIMP_GIMP_path1_pkl:  90
GIMP_GIMP_path2_pkl:  90

GIMP_path1_pkl:  90
GIMP_path2_pkl:  90
LIBHEIF_GIMP_path1_pkl:  90
LIBHEIF_GIMP_path2_pkl:  90

LIBHEIF_path1_pkl:  90
LIBHEIF_path2_pkl:  90
GIMP_LIBHEIF_path1_pkl:  90
GIMP_LIBHEIF_path2_pkl:  90


In [12]:
GIMP_csv = list(zip(GIMP_path1_csv, GIMP_path1_pkl, GIMP_path2_csv, GIMP_path2_pkl))
LIBHEIF_csv = list(zip(LIBHEIF_path1_csv, LIBHEIF_path1_pkl, LIBHEIF_path2_csv, LIBHEIF_path2_pkl))
GIMP_GIMP_csv = list(zip(GIMP_GIMP_path1_csv, GIMP_GIMP_path1_pkl, GIMP_GIMP_path2_csv, GIMP_GIMP_path2_pkl))
LIBHEIF_GIMP_csv = list(zip(LIBHEIF_GIMP_path1_csv, LIBHEIF_GIMP_path1_pkl, LIBHEIF_GIMP_path2_csv, LIBHEIF_GIMP_path2_pkl))
GIMP_LIBHEIF_csv = list(zip(GIMP_LIBHEIF_path1_csv, GIMP_LIBHEIF_path1_pkl, GIMP_LIBHEIF_path2_csv, GIMP_LIBHEIF_path2_pkl))

GIMP_GIMP_list = GIMP_csv + GIMP_GIMP_csv
LIBHEIF_GIMP_list = GIMP_csv + LIBHEIF_GIMP_csv
GIMP_LIBHEIF_list = LIBHEIF_csv + GIMP_LIBHEIF_csv


print("GIMP_GIMP_list: ", len(GIMP_GIMP_list))
print("LIBHEIF_GIMP_list: ", len(LIBHEIF_GIMP_list))
print("GIMP_LIBHEIF_list: ", len(GIMP_LIBHEIF_list))

GIMP_GIMP_list:  180
LIBHEIF_GIMP_list:  180
GIMP_LIBHEIF_list:  180


In [13]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]
luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27", "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11","LU2_25","LU2_26","LU2_27"]
chrominance_columns = ["CH1_0","CH1_1","CH1_10","CH1_26","CH1_34","CH1_36", "CH2_0","CH2_1","CH2_10","CH2_26","CH2_34","CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
train_df1_1 = pd.DataFrame(columns=pu_columns)
train_df1_2 = pd.DataFrame(columns=luminance_columns)
train_df1_3 = pd.DataFrame(columns=chrominance_columns)
train_df2 = pd.DataFrame(columns=label_columns)
train_df3 = pd.DataFrame(columns=mae1_columns)
train_df4 = pd.DataFrame(columns=mae2_columns)
train_df5 = pd.DataFrame(columns=mae_columns)
train_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in train_csv_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    train_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
        
    # print(pu_values)
        
    train_df1_1 = pd.concat([train_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    train_df1_2= pd.concat([train_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    train_df1_3 = pd.concat([train_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)

    # label_columnsの値を取得
    train_df2 = pd.concat([train_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    final_QP = extract_finalQP(train_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_old = calculate_mae(train_pkl_list[0])
    mae_d2, _ = calculate_mae(train_pkl_list[1])


    # mae_columnsの値を取得
    train_df5 = pd.concat([train_df5, pd.DataFrame({"MAE": [mae_d1_old]})], ignore_index=True)

    # final_qp_columnsの値を取得
    train_df6 = pd.concat([train_df6, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)

    # mae1_columnsの値を取得
    train_df3 = pd.concat([train_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    train_df4 = pd.concat([train_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    

# インデックスをリセット
train_df1_1.reset_index(drop=True, inplace=True)
train_df1_2.reset_index(drop=True, inplace=True)
train_df1_3.reset_index(drop=True, inplace=True)
train_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
train_df = pd.concat([train_df1_1, train_df1_2, train_df1_3, train_df3, train_df4], axis=1)
train_df_onlyGhost = pd.concat([train_df3, train_df4], axis=1)

In [14]:
# スケーラーを使って結合したデータをスケーリング
X_train = scaler.fit_transform(train_df)
X_train_onlyGhost = scaler.fit_transform(train_df_onlyGhost)

# pandasをndarrayに変換
train_df5_np = train_df5.values
FINAL_QP = train_df6.values

# ラベルの準備
Y_train = train_df2['LABEL'].astype(int)

print(f'Length of X_train: {len(X_train)}')
print(f'Length of X_train_onlyGhost: {len(X_train_onlyGhost)}')
print(f'Length of Y_train: {len(Y_train)}')
print(f'Length of train_df5_np: {len(train_df5_np)}')
print(f'Length of FINAL_QP: {len(FINAL_QP)}')

Length of X_train: 6000
Length of X_train_onlyGhost: 6000
Length of Y_train: 6000
Length of train_df5_np: 6000
Length of FINAL_QP: 6000


In [15]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]
luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27", "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11","LU2_25","LU2_26","LU2_27"]
chrominance_columns = ["CH1_0","CH1_1","CH1_10","CH1_26","CH1_34","CH1_36", "CH2_0","CH2_1","CH2_10","CH2_26","CH2_34","CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化

GIMP_GIMP_df1_1 = pd.DataFrame(columns=pu_columns)
GIMP_GIMP_df1_2 = pd.DataFrame(columns=luminance_columns)
GIMP_GIMP_df1_3 = pd.DataFrame(columns=chrominance_columns)

GIMP_GIMP_df2 = pd.DataFrame(columns=label_columns)
GIMP_GIMP_df3 = pd.DataFrame(columns=mae1_columns)
GIMP_GIMP_df4 = pd.DataFrame(columns=mae2_columns)
GIMP_GIMP_df5 = pd.DataFrame(columns=mae_columns)
GIMP_GIMP_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in GIMP_GIMP_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    test_pkl_list = [path2, path4]
    
    # pu_columnsの値を取得
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
    
    GIMP_GIMP_df1_1 = pd.concat([GIMP_GIMP_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    GIMP_GIMP_df1_2 = pd.concat([GIMP_GIMP_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    GIMP_GIMP_df1_3 = pd.concat([GIMP_GIMP_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    
    # label_columnsの値を取得
    GIMP_GIMP_df2 = pd.concat([GIMP_GIMP_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)
    
    final_QP = extract_finalQP(test_pkl_list[0])
    
    # MAEの値を取得
    mae_d1, mae_d1_old = calculate_mae(test_pkl_list[0])
    mae_d2, _ = calculate_mae(test_pkl_list[1])
    
    
    # mae_columnsの値を取得
    GIMP_GIMP_df5 = pd.concat([GIMP_GIMP_df5, pd.DataFrame({"MAE": [mae_d1_old]})], ignore_index=True)
    
    # final_qp_columnsの値を取得
    GIMP_GIMP_df6 = pd.concat([GIMP_GIMP_df6, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)
    
    # mae1_columnsの値を取得
    GIMP_GIMP_df3 = pd.concat([GIMP_GIMP_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)
    
    # mae2_columnsの値を取得
    GIMP_GIMP_df4 = pd.concat([GIMP_GIMP_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)

# インデックスをリセット
GIMP_GIMP_df1_1.reset_index(drop=True, inplace=True)
GIMP_GIMP_df1_2.reset_index(drop=True, inplace=True)
GIMP_GIMP_df1_3.reset_index(drop=True, inplace=True)
GIMP_GIMP_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
GIMP_GIMP_df = pd.concat([GIMP_GIMP_df1_1, GIMP_GIMP_df1_2, GIMP_GIMP_df1_3, GIMP_GIMP_df3, GIMP_GIMP_df4], axis=1)
GIMP_GIMP_df_onlyGhost = pd.concat([GIMP_GIMP_df3, GIMP_GIMP_df4], axis=1)

In [16]:
# スケーラーを使って結合したデータをスケーリング
X_GIMP_GIMP_test = scaler.fit_transform(GIMP_GIMP_df)
X_GIMP_GIMP_test_onlyGhost = scaler.fit_transform(GIMP_GIMP_df_onlyGhost)

# pandasをndarrayに変換
GIMP_GIMP_df5_np = GIMP_GIMP_df5.values
GIMP_GIMP_QP = GIMP_GIMP_df6.values

# ラベルの準備
Y_GIMP_GIMP_test = GIMP_GIMP_df2['LABEL'].astype(int)

print(f'Length of X_GIMP_GIMP_test: {len(X_GIMP_GIMP_test)}')
print(f'Length of X_GIMP_GIMP_test_onlyGhost: {len(X_GIMP_GIMP_test_onlyGhost)}')
print(f'Length of Y_GIMP_GIMP_test: {len(Y_GIMP_GIMP_test)}')
print(f'Length of GIMP_GIMP_df5_np: {len(GIMP_GIMP_df5_np)}')
print(f'Length of GIMP_GIMP_QP: {len(GIMP_GIMP_QP)}')

Length of X_GIMP_GIMP_test: 180
Length of X_GIMP_GIMP_test_onlyGhost: 180
Length of Y_GIMP_GIMP_test: 180
Length of GIMP_GIMP_df5_np: 180
Length of GIMP_GIMP_QP: 180


In [17]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]
luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27", "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11","LU2_25","LU2_26","LU2_27"]
chrominance_columns = ["CH1_0","CH1_1","CH1_10","CH1_26","CH1_34","CH1_36", "CH2_0","CH2_1","CH2_10","CH2_26","CH2_34","CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
LIBHEIF_GIMP_df1_1 = pd.DataFrame(columns=pu_columns)
LIBHEIF_GIMP_df1_2 = pd.DataFrame(columns=luminance_columns)
LIBHEIF_GIMP_df1_3 = pd.DataFrame(columns=chrominance_columns)

LIBHEIF_GIMP_df2 = pd.DataFrame(columns=label_columns)
LIBHEIF_GIMP_df3 = pd.DataFrame(columns=mae1_columns)
LIBHEIF_GIMP_df4 = pd.DataFrame(columns=mae2_columns)
LIBHEIF_GIMP_df5 = pd.DataFrame(columns=mae_columns)
LIBHEIF_GIMP_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in LIBHEIF_GIMP_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    test_pkl_list = [path2, path4]
    
    # pu_columnsの値を取得
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
    
    LIBHEIF_GIMP_df1_1 = pd.concat([LIBHEIF_GIMP_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    LIBHEIF_GIMP_df1_2 = pd.concat([LIBHEIF_GIMP_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    LIBHEIF_GIMP_df1_3 = pd.concat([LIBHEIF_GIMP_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
    
    # label_columnsの値を取得
    LIBHEIF_GIMP_df2 = pd.concat([LIBHEIF_GIMP_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)
    
    final_QP = extract_finalQP(test_pkl_list[0])
    
    # MAEの値を取得
    mae_d1, mae_d1_old = calculate_mae(test_pkl_list[0])
    mae_d2, _ = calculate_mae(test_pkl_list[1])
    
    
    # mae_columnsの値を取得
    LIBHEIF_GIMP_df5 = pd.concat([LIBHEIF_GIMP_df5, pd.DataFrame({"MAE": [mae_d1_old]})], ignore_index=True)
    
    # final_qp_columnsの値を取得
    LIBHEIF_GIMP_df6 = pd.concat([LIBHEIF_GIMP_df6, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)
    
    # mae1_columnsの値を取得
    LIBHEIF_GIMP_df3 = pd.concat([LIBHEIF_GIMP_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)
    
    # mae2_columnsの値を取得
    LIBHEIF_GIMP_df4 = pd.concat([LIBHEIF_GIMP_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)

# インデックスをリセット
LIBHEIF_GIMP_df1_1.reset_index(drop=True, inplace=True)
LIBHEIF_GIMP_df1_2.reset_index(drop=True, inplace=True)
LIBHEIF_GIMP_df1_3.reset_index(drop=True, inplace=True)
LIBHEIF_GIMP_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
LIBHEIF_GIMP_df = pd.concat([LIBHEIF_GIMP_df1_1, LIBHEIF_GIMP_df1_2, LIBHEIF_GIMP_df1_3, LIBHEIF_GIMP_df3, LIBHEIF_GIMP_df4], axis=1)
LIBHEIF_GIMP_df_onlyGhost = pd.concat([LIBHEIF_GIMP_df3, LIBHEIF_GIMP_df4], axis=1)

In [18]:
# スケーラーを使って結合したデータをスケーリング
X_LIBHEIF_GIMP_test = scaler.fit_transform(LIBHEIF_GIMP_df)
X_LIBHEIF_GIMP_test_onlyGhost = scaler.fit_transform(LIBHEIF_GIMP_df_onlyGhost)

# pandasをndarrayに変換
LIBHEIF_GIMP_df5_np = LIBHEIF_GIMP_df5.values
LIBHEIF_GIMP_QP = LIBHEIF_GIMP_df6.values

# ラベルの準備
Y_LIBHEIF_GIMP_test = LIBHEIF_GIMP_df2['LABEL'].astype(int)

print(f'Length of X_LIBHEIF_GIMP_test: {len(X_LIBHEIF_GIMP_test)}')
print(f'Length of X_LIBHEIF_GIMP_test_onlyGhost: {len(X_LIBHEIF_GIMP_test_onlyGhost)}')
print(f'Length of Y_LIBHEIF_GIMP_test: {len(Y_LIBHEIF_GIMP_test)}')
print(f'Length of LIBHEIF_GIMP_df5_np: {len(LIBHEIF_GIMP_df5_np)}')
print(f'Length of LIBHEIF_GIMP_QP: {len(LIBHEIF_GIMP_QP)}')
print()

Length of X_LIBHEIF_GIMP_test: 180
Length of X_LIBHEIF_GIMP_test_onlyGhost: 180
Length of Y_LIBHEIF_GIMP_test: 180
Length of LIBHEIF_GIMP_df5_np: 180
Length of LIBHEIF_GIMP_QP: 180



In [19]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]
luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27", "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11","LU2_25","LU2_26","LU2_27"]
chrominance_columns = ["CH1_0","CH1_1","CH1_10","CH1_26","CH1_34","CH1_36", "CH2_0","CH2_1","CH2_10","CH2_26","CH2_34","CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
GIMP_LIBHEIF_df1_1 = pd.DataFrame(columns=pu_columns)
GIMP_LIBHEIF_df1_2 = pd.DataFrame(columns=luminance_columns)
GIMP_LIBHEIF_df1_3 = pd.DataFrame(columns=chrominance_columns)

GIMP_LIBHEIF_df2 = pd.DataFrame(columns=label_columns)
GIMP_LIBHEIF_df3 = pd.DataFrame(columns=mae1_columns)
GIMP_LIBHEIF_df4 = pd.DataFrame(columns=mae2_columns)
GIMP_LIBHEIF_df5 = pd.DataFrame(columns=mae_columns)
GIMP_LIBHEIF_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in GIMP_LIBHEIF_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    test_pkl_list = [path2, path4]
    
    # pu_columnsの値を取得
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
    
    GIMP_LIBHEIF_df1_1 = pd.concat([GIMP_LIBHEIF_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    GIMP_LIBHEIF_df1_2 = pd.concat([GIMP_LIBHEIF_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    GIMP_LIBHEIF_df1_3 = pd.concat([GIMP_LIBHEIF_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
    
    # label_columnsの値を取得
    GIMP_LIBHEIF_df2 = pd.concat([GIMP_LIBHEIF_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)
    
    final_QP = extract_finalQP(test_pkl_list[0])
    
    # MAEの値を取得
    mae_d1, mae_d1_old = calculate_mae(test_pkl_list[0])
    mae_d2, _ = calculate_mae(test_pkl_list[1])
    
    
    # mae_columnsの値を取得
    GIMP_LIBHEIF_df5 = pd.concat([GIMP_LIBHEIF_df5, pd.DataFrame({"MAE": [mae_d1_old]})], ignore_index=True)
    
    # final_qp_columnsの値を取得
    GIMP_LIBHEIF_df6 = pd.concat([GIMP_LIBHEIF_df6, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)
    
    # mae1_columnsの値を取得
    GIMP_LIBHEIF_df3 = pd.concat([GIMP_LIBHEIF_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)
    
    # mae2_columnsの値を取得
    GIMP_LIBHEIF_df4 = pd.concat([GIMP_LIBHEIF_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)

# インデックスをリセット
GIMP_LIBHEIF_df1_1.reset_index(drop=True, inplace=True)
GIMP_LIBHEIF_df1_2.reset_index(drop=True, inplace=True)
GIMP_LIBHEIF_df1_3.reset_index(drop=True, inplace=True)
GIMP_LIBHEIF_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
GIMP_LIBHEIF_df = pd.concat([GIMP_LIBHEIF_df1_1, GIMP_LIBHEIF_df1_2, GIMP_LIBHEIF_df1_3, GIMP_LIBHEIF_df3, GIMP_LIBHEIF_df4], axis=1)
GIMP_LIBHEIF_df_onlyGhost = pd.concat([GIMP_LIBHEIF_df3, GIMP_LIBHEIF_df4], axis=1)

In [20]:
# スケーラーを使って結合したデータをスケーリング
X_GIMP_LIBHEIF_test = scaler.fit_transform(GIMP_LIBHEIF_df)
X_GIMP_LIBHEIF_test_onlyGhost = scaler.fit_transform(GIMP_LIBHEIF_df_onlyGhost)

# pandasをndarrayに変換
GIMP_LIBHEIF_df5_np = GIMP_LIBHEIF_df5.values
GIMP_LIBHEIF_QP = GIMP_LIBHEIF_df6.values

# ラベルの準備
Y_GIMP_LIBHEIF_test = GIMP_LIBHEIF_df2['LABEL'].astype(int)

print(f'Length of X_GIMP_LIBHEIF_test: {len(X_GIMP_LIBHEIF_test)}')
print(f'Length of X_GIMP_LIBHEIF_test_onlyGhost: {len(X_GIMP_LIBHEIF_test_onlyGhost)}')
print(f'Length of Y_GIMP_LIBHEIF_test: {len(Y_GIMP_LIBHEIF_test)}')
print(f'Length of GIMP_LIBHEIF_df5_np: {len(GIMP_LIBHEIF_df5_np)}')
print(f'Length of GIMP_LIBHEIF_QP: {len(GIMP_LIBHEIF_QP)}')


Length of X_GIMP_LIBHEIF_test: 180
Length of X_GIMP_LIBHEIF_test_onlyGhost: 180
Length of Y_GIMP_LIBHEIF_test: 180
Length of GIMP_LIBHEIF_df5_np: 180
Length of GIMP_LIBHEIF_QP: 180


In [21]:
plt.rcParams["font.size"]=5
plt.rcParams["figure.figsize"]=(2.0, 1.0)
plt.rcParams["figure.dpi"]= 300


# Cの範囲を指定
C_values = {'C': [0.01, 0.1, 1, 10, 100, 1000, 2000, 3000, 4000, 5000]}
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 結果のデータフレームを初期化
results = pd.DataFrame(columns=['C_RBF_GIMP_GIMP','GIMP_GIMP_RBF','C_RBF_LIBHEIF_GIMP','LIBHEIF_GIMP_RBF',
                                'C_RBF_GIMP_LIBHEIF','GIMP_LIBHEIF_RBF','C_LINEAR_GIMP_GIMP','GIMP_GIMP_LINEAR',
                                'C_LINEAR_LIBHEIF_GIMP', 'LIBHEIF_GIMP_LINEAR','C_LINEAR_GIMP_LIBHEIF', 'GIMP_LIBHEIF_LINEAR',
                                'C_onlyGhost_RBF_GIMP_GIMP', 'GIMP_GIMP_onlyGhost_RBF','C_onlyGhost_RBF_LIBHEIF_GIMP', 'LIBHEIF_GIMP_onlyGhost_RBF',
                                'C_onlyGhost_RBF_GIMP_LIBHEIF', 'GIMP_LIBHEIF_onlyGhost_RBF','C_onlyGhost_LINEAR_GIMP_GIMP', 'GIMP_GIMP_onlyGhost_LINEAR',
                                'C_onlyGhost_LINEAR_LIBHEIF_GIMP', 'LIBHEIF_GIMP_onlyGhost_LINEAR','C_onlyGhost_LINEAR_GIMP_LIBHEIF', 'GIMP_LIBHEIF_onlyGhost_LINEAR',
                                'Threshold_GIMP_GIMP', 'GIMP_GIMP_old','Threshold_LIBHEIF_GIMP', 'LIBHEIF_GIMP_old',
                                'Threshold_GIMP_LIBHEIF', 'GIMP_LIBHEIF_old'])


# 訓練用データ（RBFとlinearカーネル用）
original_X_train, original_X_train_onlyGhost = X_train, X_train_onlyGhost
original_Y_train = Y_train

# テストデータ(GIMP-GIMP, LIBHEIF_GIMP, GIMP_LIBHEIF)
original_X_GIMP_GIMP_test, original_X_GIMP_GIMP_test_onlyGhost = X_GIMP_GIMP_test, X_GIMP_GIMP_test_onlyGhost
original_X_LIBHEIF_GIMP_test, original_X_LIBHEIF_GIMP_test_onlyGhost = X_LIBHEIF_GIMP_test, X_LIBHEIF_GIMP_test_onlyGhost
original_X_GIMP_LIBHEIF_test, original_X_GIMP_LIBHEIF_test_onlyGhost = X_GIMP_LIBHEIF_test, X_GIMP_LIBHEIF_test_onlyGhost

original_Y_GIMP_GIMP_test = Y_GIMP_GIMP_test
original_Y_LIBHEIF_GIMP_test = Y_LIBHEIF_GIMP_test
original_Y_GIMP_LIBHEIF_test = Y_GIMP_LIBHEIF_test

# 閾値用テストデータ(GIMP-GIMP, LIBHEIF_GIMP, GIMP_LIBHEIF)
original_GIMP_GIMP_old, original_GIMP_GIMP_final_QP = GIMP_GIMP_df5_np, GIMP_GIMP_QP
original_LIBHEIF_GIMP_old, original_LIBHEIF_GIMP_final_QP = LIBHEIF_GIMP_df5_np, LIBHEIF_GIMP_QP
original_GIMP_LIBHEIF_old, original_GIMP_LIBHEIF_final_QP = GIMP_LIBHEIF_df5_np, GIMP_LIBHEIF_QP


# k-fold cross-validation
for fold, (train_ids, test_ids) in enumerate(kfold.split(original_X_train, original_Y_train)):
    print(f"<Fold-{fold+1}>")
    print()
    
    results_old = []
    
    X_train_val, _ = original_X_train[train_ids], original_X_train[test_ids]
    X_train_onlyGhost_val, _ = original_X_train_onlyGhost[train_ids], original_X_train_onlyGhost[test_ids]
    
    X_GIMP_GIMP_test, X_GIMP_GIMP_test_onlyGhost = original_X_GIMP_GIMP_test, original_X_GIMP_GIMP_test_onlyGhost
    X_LIBHEIF_GIMP_test, X_LIBHEIF_GIMP_test_onlyGhost = original_X_LIBHEIF_GIMP_test, original_X_LIBHEIF_GIMP_test_onlyGhost
    X_GIMP_LIBHEIF_test, X_GIMP_LIBHEIF_test_onlyGhost = original_X_GIMP_LIBHEIF_test, original_X_GIMP_LIBHEIF_test_onlyGhost
    
    X_GIMP_GIMP_old, GIMP_GIMP_QP = original_GIMP_GIMP_old, original_GIMP_GIMP_final_QP
    X_LIBHEIF_GIMP_old, LIBHEIF_GIMP_QP = original_LIBHEIF_GIMP_old, original_LIBHEIF_GIMP_final_QP
    X_GIMP_LIBHEIF_old, GIMP_LIBHEIF_QP = original_GIMP_LIBHEIF_old, original_GIMP_LIBHEIF_final_QP
    
    
    # 全体を訓練・検証ラベルとテストラベルに分割
    Y_train_val, _ = original_Y_train[train_ids], original_Y_train[test_ids]
    
    Y_GIMP_GIMP_test = original_Y_GIMP_GIMP_test
    Y_LIBHEIF_GIMP_test = original_Y_LIBHEIF_GIMP_test
    Y_GIMP_LIBHEIF_test = original_Y_GIMP_LIBHEIF_test
    
    # 訓練・検証データ（ラベル）を訓練データ（ラベル）と検証データ（ラベル）に分割
    X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=600, random_state=42)
    X_train_onlyGhost, X_val_onlyGhost, Y_train, Y_val = train_test_split(X_train_onlyGhost_val, Y_train_val, test_size=600, random_state=42)
    
 
    GIMP_GIMP_best_threshold = 0
    GIMP_GIMP_best_accuracy = 0
    GIMP_GIMP_best_predicted_labels = []
    GIMP_GIMP_best_ground_truth_labels = []
    
    LIBHEIF_GIMP_best_threshold = 0
    LIBHEIF_GIMP_best_accuracy = 0
    LIBHEIF_GIMP_best_predicted_labels = []
    LIBHEIF_GIMP_best_ground_truth_labels = []
    
    GIMP_LIBHEIF_best_threshold = 0
    GIMP_LIBHEIF_best_accuracy = 0
    GIMP_LIBHEIF_best_predicted_labels = []
    GIMP_LIBHEIF_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        GIMP_GIMP_old = [is_double_compressed(X_GIMP_GIMP_old[i], GIMP_GIMP_QP[i], threshold) for i in range(180)]
        
        GIMP_GIMP_predicted_labels = [int(is_double) for is_double in GIMP_GIMP_old]
        GIMP_GIMP_ground_truth_labels = [label for label in Y_GIMP_GIMP_test]
        GIMP_GIMP_accuracy = sum(1 for true_label, pred_label in zip(GIMP_GIMP_ground_truth_labels, GIMP_GIMP_predicted_labels) if true_label == pred_label) / len(GIMP_GIMP_ground_truth_labels)
        
        if GIMP_GIMP_accuracy > GIMP_GIMP_best_accuracy:
            GIMP_GIMP_best_accuracy = GIMP_GIMP_accuracy
            GIMP_GIMP_best_threshold = threshold
            GIMP_GIMP_best_predicted_labels = GIMP_GIMP_predicted_labels
            GIMP_GIMP_best_ground_truth_labels = GIMP_GIMP_ground_truth_labels
            
            
    for threshold in np.arange(0.01,1.01,0.01):
        LIBHEIF_GIMP_old = [is_double_compressed(X_LIBHEIF_GIMP_old[i], LIBHEIF_GIMP_QP[i], threshold) for i in range(180)]
        
        LIBHEIF_GIMP_predicted_labels = [int(is_double) for is_double in LIBHEIF_GIMP_old]
        LIBHEIF_GIMP_ground_truth_labels = [label for label in Y_LIBHEIF_GIMP_test]
        LIBHEIF_GIMP_accuracy = sum(1 for true_label, pred_label in zip(LIBHEIF_GIMP_ground_truth_labels, LIBHEIF_GIMP_predicted_labels) if true_label == pred_label) / len(LIBHEIF_GIMP_ground_truth_labels)
        
        if LIBHEIF_GIMP_accuracy > LIBHEIF_GIMP_best_accuracy:
            LIBHEIF_GIMP_best_accuracy = LIBHEIF_GIMP_accuracy
            LIBHEIF_GIMP_best_threshold = threshold
            LIBHEIF_GIMP_best_predicted_labels = LIBHEIF_GIMP_predicted_labels
            LIBHEIF_GIMP_best_ground_truth_labels = LIBHEIF_GIMP_ground_truth_labels
            
            
    for threshold in np.arange(0.01,1.01,0.01):
        GIMP_LIBHEIF_old = [is_double_compressed(X_GIMP_LIBHEIF_old[i], GIMP_LIBHEIF_QP[i], threshold) for i in range(180)]
        
        GIMP_LIBHEIF_predicted_labels = [int(is_double) for is_double in GIMP_LIBHEIF_old]
        GIMP_LIBHEIF_ground_truth_labels = [label for label in Y_GIMP_LIBHEIF_test]
        GIMP_LIBHEIF_accuracy = sum(1 for true_label, pred_label in zip(GIMP_LIBHEIF_ground_truth_labels, GIMP_LIBHEIF_predicted_labels) if true_label == pred_label) / len(GIMP_LIBHEIF_ground_truth_labels)
        
        if GIMP_LIBHEIF_accuracy > GIMP_LIBHEIF_best_accuracy:
            GIMP_LIBHEIF_best_accuracy = GIMP_LIBHEIF_accuracy
            GIMP_LIBHEIF_best_threshold = threshold
            GIMP_LIBHEIF_best_predicted_labels = GIMP_LIBHEIF_predicted_labels
            GIMP_LIBHEIF_best_ground_truth_labels = GIMP_LIBHEIF_ground_truth_labels
            
    
    best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = 0, None, None
    best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = 0, None, None
    
    best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = 0, None, None
    best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = 0, None, None
        
    for C_value in C_values['C']:    
        # SVMモデルのインスタンスを作成
        svm_model_RBF = SVC(kernel='rbf', C=C_value)
        svm_model_onlyGhost_RBF = SVC(kernel='rbf', C=C_value)
        
        svm_model_LINEAR = SVC(kernel='linear', C=C_value)
        svm_model_onlyGhost_LINEAR = SVC(kernel='linear', C=C_value)

        # 訓練データで訓練
        svm_model_RBF.fit(X_train, Y_train)
        svm_model_onlyGhost_RBF.fit(X_train_onlyGhost, Y_train)
        
        svm_model_LINEAR.fit(X_train, Y_train)
        svm_model_onlyGhost_LINEAR.fit(X_train_onlyGhost, Y_train)

        
        val_accuracy_RBF = accuracy_score(Y_val, svm_model_RBF.predict(X_val))
        val_accuracy_onlyGhost_RBF = accuracy_score(Y_val, svm_model_onlyGhost_RBF.predict(X_val_onlyGhost))
        
        val_accuracy_LINEAR = accuracy_score(Y_val, svm_model_LINEAR.predict(X_val))
        val_accuracy_onlyGhost_LINEAR = accuracy_score(Y_val, svm_model_onlyGhost_LINEAR.predict(X_val_onlyGhost))
        

        # 検証データでの精度が最も高かった場合、そのモデルを保存
        if val_accuracy_RBF > best_val_score_RBF:
            best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = val_accuracy_RBF, svm_model_RBF, C_value

        if val_accuracy_onlyGhost_RBF > best_val_score_onlyGhost_RBF:
            best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = val_accuracy_onlyGhost_RBF, svm_model_onlyGhost_RBF, C_value
            
        if val_accuracy_LINEAR > best_val_score_LINEAR:
            best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = val_accuracy_LINEAR, svm_model_LINEAR, C_value

        if val_accuracy_onlyGhost_LINEAR > best_val_score_onlyGhost_LINEAR:
            best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = val_accuracy_onlyGhost_LINEAR, svm_model_onlyGhost_LINEAR, C_value

            
    # テストデータで評価
    GIMP_GIMP_predictions_RBF = best_svm_model_RBF.predict(X_GIMP_GIMP_test)
    GIMP_GIMP_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_GIMP_GIMP_test)
    GIMP_GIMP_accuracy_RBF = accuracy_score(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_RBF)
    GIMP_GIMP_report_RBF = classification_report(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_RBF)
    print(f'GIMP_GIMP_report_RBF:\n{GIMP_GIMP_report_RBF}')
    
    LIBHEIF_GIMP_predictions_RBF = best_svm_model_RBF.predict(X_LIBHEIF_GIMP_test)
    LIBHEIF_GIMP_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_LIBHEIF_GIMP_test)
    LIBHEIF_GIMP_accuracy_RBF = accuracy_score(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_RBF)
    LIBHEIF_GIMP_report_RBF = classification_report(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_RBF)
    print(f'LIBHEIF_GIMP_report_RBF:\n{LIBHEIF_GIMP_report_RBF}')
    
    GIMP_LIBHEIF_predictions_RBF = best_svm_model_RBF.predict(X_GIMP_LIBHEIF_test)
    GIMP_LIBHEIF_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_GIMP_LIBHEIF_test)
    GIMP_LIBHEIF_accuracy_RBF = accuracy_score(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_RBF)
    GIMP_LIBHEIF_report_RBF = classification_report(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_RBF)
    print(f'GIMP_LIBHEIF_report_RBF:\n{GIMP_LIBHEIF_report_RBF}')
    
    GIMP_GIMP_predictions_LINEAR = best_svm_model_LINEAR.predict(X_GIMP_GIMP_test)
    GIMP_GIMP_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_GIMP_GIMP_test)
    GIMP_GIMP_accuracy_LINEAR = accuracy_score(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_LINEAR)
    GIMP_GIMP_report_LINEAR = classification_report(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_LINEAR)
    print(f'GIMP_GIMP_report_LINEAR:\n{GIMP_GIMP_report_LINEAR}')
    
    LIBHEIF_GIMP_predictions_LINEAR = best_svm_model_LINEAR.predict(X_LIBHEIF_GIMP_test)
    LIBHEIF_GIMP_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_LIBHEIF_GIMP_test)
    LIBHEIF_GIMP_accuracy_LINEAR = accuracy_score(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_LINEAR)
    LIBHEIF_GIMP_report_LINEAR = classification_report(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_LINEAR)
    print(f'LIBHEIF_GIMP_report_LINEAR:\n{LIBHEIF_GIMP_report_LINEAR}')
    
    GIMP_LIBHEIF_predictions_LINEAR = best_svm_model_LINEAR.predict(X_GIMP_LIBHEIF_test)
    GIMP_LIBHEIF_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_GIMP_LIBHEIF_test)
    GIMP_LIBHEIF_accuracy_LINEAR = accuracy_score(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_LINEAR)
    GIMP_LIBHEIF_report_LINEAR = classification_report(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_LINEAR)
    print(f'GIMP_LIBHEIF_report_LINEAR:\n{GIMP_LIBHEIF_report_LINEAR}')
    
    
    # テストデータで評価
    GIMP_GIMP_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_GIMP_GIMP_test_onlyGhost)
    GIMP_GIMP_predictions_prob_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.decision_function(X_GIMP_GIMP_test_onlyGhost)
    GIMP_GIMP_accuracy_onlyGhost_RBF = accuracy_score(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_onlyGhost_RBF)
    GIMP_GIMP_report_onlyGhost_RBF = classification_report(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_onlyGhost_RBF)
    print(f'GIMP_GIMP_report_onlyGhost_RBF:\n{GIMP_GIMP_report_onlyGhost_RBF}')
    
    LIBHEIF_GIMP_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_LIBHEIF_GIMP_test_onlyGhost)
    LIBHEIF_GIMP_predictions_prob_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.decision_function(X_LIBHEIF_GIMP_test_onlyGhost)
    LIBHEIF_GIMP_accuracy_onlyGhost_RBF = accuracy_score(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_onlyGhost_RBF)
    LIBHEIF_GIMP_report_onlyGhost_RBF = classification_report(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_onlyGhost_RBF)
    print(f'LIBHEIF_GIMP_report_onlyGhost_RBF:\n{LIBHEIF_GIMP_report_onlyGhost_RBF}')
    
    GIMP_LIBHEIF_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_GIMP_LIBHEIF_test_onlyGhost)
    GIMP_LIBHEIF_predictions_prob_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.decision_function(X_GIMP_LIBHEIF_test_onlyGhost)
    GIMP_LIBHEIF_accuracy_onlyGhost_RBF = accuracy_score(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_onlyGhost_RBF)
    GIMP_LIBHEIF_report_onlyGhost_RBF = classification_report(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_onlyGhost_RBF)
    print(f'GIMP_LIBHEIF_report_onlyGhost_RBF:\n{GIMP_LIBHEIF_report_onlyGhost_RBF}')
    
    GIMP_GIMP_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_GIMP_GIMP_test_onlyGhost)
    GIMP_GIMP_predictions_prob_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.decision_function(X_GIMP_GIMP_test_onlyGhost)
    GIMP_GIMP_accuracy_onlyGhost_LINEAR = accuracy_score(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_onlyGhost_LINEAR)
    GIMP_GIMP_report_onlyGhost_LINEAR = classification_report(Y_GIMP_GIMP_test, GIMP_GIMP_predictions_onlyGhost_LINEAR)
    print(f'GIMP_GIMP_report_onlyGhost_LINEAR:\n{GIMP_GIMP_report_onlyGhost_LINEAR}')
    
    LIBHEIF_GIMP_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_LIBHEIF_GIMP_test_onlyGhost)
    LIBHEIF_GIMP_predictions_prob_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.decision_function(X_LIBHEIF_GIMP_test_onlyGhost)
    LIBHEIF_GIMP_accuracy_onlyGhost_LINEAR = accuracy_score(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_onlyGhost_LINEAR)
    LIBHEIF_GIMP_report_onlyGhost_LINEAR = classification_report(Y_LIBHEIF_GIMP_test, LIBHEIF_GIMP_predictions_onlyGhost_LINEAR)
    print(f'LIBHEIF_GIMP_report_onlyGhost_LINEAR:\n{LIBHEIF_GIMP_report_onlyGhost_LINEAR}')
    
    GIMP_LIBHEIF_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_GIMP_LIBHEIF_test_onlyGhost)
    GIMP_LIBHEIF_predictions_prob_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.decision_function(X_GIMP_LIBHEIF_test_onlyGhost)
    GIMP_LIBHEIF_accuracy_onlyGhost_LINEAR = accuracy_score(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_onlyGhost_LINEAR)
    GIMP_LIBHEIF_report_onlyGhost_LINEAR = classification_report(Y_GIMP_LIBHEIF_test, GIMP_LIBHEIF_predictions_onlyGhost_LINEAR)
    print(f'GIMP_LIBHEIF_report_onlyGhost_LINEAR:\n{GIMP_LIBHEIF_report_onlyGhost_LINEAR}')
    

    GIMP_GIMP_old = classification_report(GIMP_GIMP_best_ground_truth_labels, GIMP_GIMP_best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0)
    print(f'GIMP_GIMP_old:\n{GIMP_GIMP_old}')
    
    LIBHEIF_GIMP_old = classification_report(LIBHEIF_GIMP_best_ground_truth_labels, LIBHEIF_GIMP_best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0)
    print(f'LIBHEIF_GIMP_old:\n{LIBHEIF_GIMP_old}')
    
    GIMP_LIBHEIF_old = classification_report(GIMP_LIBHEIF_best_ground_truth_labels, GIMP_LIBHEIF_best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0)
    print(f'GIMP_LIBHEIF_old:\n{GIMP_LIBHEIF_old}')
        
    # Test結果を保存
    
    result_row = {'C_RBF_GIMP_GIMP': best_c_value_RBF, 'GIMP_GIMP_RBF': GIMP_GIMP_accuracy_RBF,
                  'C_RBF_LIBHEIF_GIMP': best_c_value_RBF, 'LIBHEIF_GIMP_RBF': LIBHEIF_GIMP_accuracy_RBF,
                  'C_RBF_GIMP_LIBHEIF': best_c_value_RBF, 'GIMP_LIBHEIF_RBF': GIMP_LIBHEIF_accuracy_RBF,
                    
                  'C_LINEAR_GIMP_GIMP': best_c_value_LINEAR, 'GIMP_GIMP_LINEAR': GIMP_GIMP_accuracy_LINEAR,
                  'C_LINEAR_LIBHEIF_GIMP': best_c_value_LINEAR, 'LIBHEIF_GIMP_LINEAR': LIBHEIF_GIMP_accuracy_LINEAR,
                  'C_LINEAR_GIMP_LIBHEIF': best_c_value_LINEAR, 'GIMP_LIBHEIF_LINEAR': GIMP_LIBHEIF_accuracy_LINEAR,
                  
                  'C_onlyGhost_RBF_GIMP_GIMP': best_c_value_onlyGhost_RBF, 'GIMP_GIMP_onlyGhost_RBF': GIMP_GIMP_accuracy_onlyGhost_RBF,
                  'C_onlyGhost_RBF_LIBHEIF_GIMP': best_c_value_onlyGhost_RBF, 'LIBHEIF_GIMP_onlyGhost_RBF': LIBHEIF_GIMP_accuracy_onlyGhost_RBF,
                  'C_onlyGhost_RBF_GIMP_LIBHEIF': best_c_value_onlyGhost_RBF, 'GIMP_LIBHEIF_onlyGhost_RBF': GIMP_LIBHEIF_accuracy_onlyGhost_RBF,
                  
                  'C_onlyGhost_LINEAR_GIMP_GIMP': best_c_value_onlyGhost_LINEAR, 'GIMP_GIMP_onlyGhost_LINEAR': GIMP_GIMP_accuracy_onlyGhost_LINEAR,
                  'C_onlyGhost_LINEAR_LIBHEIF_GIMP': best_c_value_onlyGhost_LINEAR, 'LIBHEIF_GIMP_onlyGhost_LINEAR': LIBHEIF_GIMP_accuracy_onlyGhost_LINEAR,
                  'C_onlyGhost_LINEAR_GIMP_LIBHEIF': best_c_value_onlyGhost_LINEAR, 'GIMP_LIBHEIF_onlyGhost_LINEAR': GIMP_LIBHEIF_accuracy_onlyGhost_LINEAR,
                  
                  'Threshold_GIMP_GIMP': GIMP_GIMP_best_threshold, 'GIMP_GIMP_old': GIMP_GIMP_best_accuracy,
                  'Threshold_LIBHEIF_GIMP': LIBHEIF_GIMP_best_threshold, 'LIBHEIF_GIMP_old': LIBHEIF_GIMP_best_accuracy,
                  'Threshold_GIMP_LIBHEIF': GIMP_LIBHEIF_best_threshold, 'GIMP_LIBHEIF_old': GIMP_LIBHEIF_best_accuracy,
                 
                 }

    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)

# 結果を表示
print(results)

<Fold-1>

GIMP_GIMP_report_RBF:
              precision    recall  f1-score   support

           0       0.68      0.56      0.61        90
           1       0.63      0.74      0.68        90

    accuracy                           0.65       180
   macro avg       0.66      0.65      0.65       180
weighted avg       0.66      0.65      0.65       180

LIBHEIF_GIMP_report_RBF:
              precision    recall  f1-score   support

           0       0.63      0.56      0.59        90
           1       0.60      0.68      0.64        90

    accuracy                           0.62       180
   macro avg       0.62      0.62      0.62       180
weighted avg       0.62      0.62      0.62       180

GIMP_LIBHEIF_report_RBF:
              precision    recall  f1-score   support

           0       0.67      0.68      0.67        90
           1       0.67      0.67      0.67        90

    accuracy                           0.67       180
   macro avg       0.67      0.67      0.67   

In [22]:
def print_stats(column_name, label):
    average = round(results[column_name].mean(), 2)
    std_dev = round(results[column_name].std(), 2)
    max_value = round(results[column_name].max(), 2)
    min_value = round(results[column_name].min(), 2)

    print(f'Average Test Score {column_name}{label}: {average}')
    print(f'Standard Deviation of Test Score {column_name}{label}: {std_dev}')
    print(f'Maximum Test Score {column_name}{label}: {max_value}')
    print(f'Minimum Test Score {column_name}{label}: {min_value}')
    print()

# 'Test_Score'列に関して統計情報を表示
print_stats('GIMP_GIMP_RBF', 'with RBF')
print_stats('LIBHEIF_GIMP_RBF', 'with RBF')
print_stats('GIMP_LIBHEIF_RBF', 'with RBF')

print_stats('GIMP_GIMP_LINEAR', 'with LINEAR')
print_stats('LIBHEIF_GIMP_LINEAR', 'with LINEAR')
print_stats('GIMP_LIBHEIF_LINEAR', 'with LINEAR')

# 'Test_Score_onlyGhost'列に関して統計情報を表示
print_stats('GIMP_GIMP_onlyGhost_RBF', 'with only Ghost and RBF')
print_stats('LIBHEIF_GIMP_onlyGhost_RBF', 'with only Ghost and RBF')
print_stats('GIMP_LIBHEIF_onlyGhost_RBF', 'with only Ghost and RBF')

print_stats('GIMP_GIMP_onlyGhost_LINEAR', 'with only Ghost and LINEAR')
print_stats('LIBHEIF_GIMP_onlyGhost_LINEAR', 'with only Ghost and LINEAR')
print_stats('GIMP_LIBHEIF_onlyGhost_LINEAR', 'with only Ghost and LINEAR')

# 'Test__old'列に関して統計情報を表示
print_stats('GIMP_GIMP_old', 'with old model')
print_stats('LIBHEIF_GIMP_old', 'with old model')
print_stats('GIMP_LIBHEIF_old', 'with old model')

Average Test Score GIMP_GIMP_RBFwith RBF: 0.64
Standard Deviation of Test Score GIMP_GIMP_RBFwith RBF: 0.05
Maximum Test Score GIMP_GIMP_RBFwith RBF: 0.72
Minimum Test Score GIMP_GIMP_RBFwith RBF: 0.59

Average Test Score LIBHEIF_GIMP_RBFwith RBF: 0.6
Standard Deviation of Test Score LIBHEIF_GIMP_RBFwith RBF: 0.03
Maximum Test Score LIBHEIF_GIMP_RBFwith RBF: 0.67
Minimum Test Score LIBHEIF_GIMP_RBFwith RBF: 0.57

Average Test Score GIMP_LIBHEIF_RBFwith RBF: 0.66
Standard Deviation of Test Score GIMP_LIBHEIF_RBFwith RBF: 0.03
Maximum Test Score GIMP_LIBHEIF_RBFwith RBF: 0.73
Minimum Test Score GIMP_LIBHEIF_RBFwith RBF: 0.63

Average Test Score GIMP_GIMP_LINEARwith LINEAR: 0.64
Standard Deviation of Test Score GIMP_GIMP_LINEARwith LINEAR: 0.03
Maximum Test Score GIMP_GIMP_LINEARwith LINEAR: 0.69
Minimum Test Score GIMP_GIMP_LINEARwith LINEAR: 0.61

Average Test Score LIBHEIF_GIMP_LINEARwith LINEAR: 0.6
Standard Deviation of Test Score LIBHEIF_GIMP_LINEARwith LINEAR: 0.02
Maximum Test Sco